In [2]:
!pip install WebSocket-client

     |████████████████████████████████| 71kB 3.7MB/s 


In [3]:
import websocket
import json
import numpy as np

In [30]:
cc = 'btcusd'
interval = '1m'

# socket = f'wss://stream.binance.com:9443/ws/{cc}t@kline_{interval}'
socket = f"wss://fstream.binance.com:9443/ws/{cc}@markPrice"

In [31]:
# Trading Strategy Parameters
aroon_time_period = 5

amount = 1000
core_trade_amount = amount*0.80
core_quantity = 0
trade_amount = amount*0.20
core_to_trade = True

portfolio = 0
investment, real_time_portfolio_value, closes, highs, lows = [], [], [], [], []
money_end = amount

In [32]:
# Buying and Selling functions

def buy(allocated_money, price):
  global portfolio, money_end
  quantity = allocated_money/price
  money_end -= quantity*price
  portfolio += quantity
  if investment == []:
    investment.append(allocated_money)
  else:
    investment.append(allocated_money)
    investment[-1] += investment[-2]

def sell(allocated_money, price):
  global money_end, portfolio
  quantity = allocated_money / price
  money_end += allocated_money
  portfolio -= quantity
  investment.append(-allocated_money)
  investment[-1] += investment[-2]

In [33]:
# Bitcoin Bot

def on_close(ws):
  port_value = portfolio*closes[-1]
  if port_value > 0:
    sell(port_value,price = closes[-1])
  else:
    buy(-port_value, price = closes[-1])
  money_end += investment[-1]
  print('All trades settled')

def on_message(ws,message):
  global portfolio, investment, closes, highs, lows, money_end, core_to_trade, core_quantity, real_time_portfolio_value
  json_message = json.loads(message)
  # print(json_message)
  cs = json_message['k']
  candle_closed, close, high, low = cs['x'], cs['c'], cs['h'], cs['l']
  # print(candle_close, close, high)

  if candle_closed:
    closes.append(float(close))
    highs.append(float(high))
    lows.append(float(low))
    last_price = closes[-1]
    print(f'Closes: {closes}')
    
    if core_to_trade:
      buy(core_trade_amount, price=closes[-1])
      print(f'Core Investment: We bought ${core_trade_amount} worth of bitcoin', '\n')
      core_quantity += core_trade_amount/closes[-1]
      core_to_trade = False

    aroon = talib.AROONOSC(np.array(highs), np.array(lows), aroon_time_period)
    print(aroon, "arroooon value")
    last_aroon = round(aroon[-1],2)
    amt = last_aroon*trade_amount/100
    port_value = portfolio*last_price - core_quantity*last_price
    trade_amt = amt - port_value
    RT_portfolio_value = money_end + port_value + core_quantity*last_price
    real_time_portfolio_value.append(float(RT_portfolio_value))
    print(f'The Last Aroon is "{last_aroon}" and recommended exposure is "${amt}"')
    print(f'Real-Time Portfolio Value: ${RT_portfolio_value}', '\n')
    
    if trade_amt > 0:
      buy(trade_amt, price=last_price)
      print(f'We bought ${trade_amt} worth of bitcoin', '\n', '\n')
    elif trade_amt < 0:
      sell(-trade_amt, price=last_price)
      print(f'We sold ${-trade_amt} worth of bitcoin', '\n', '\n')

ws = websocket.WebSocketApp(socket, on_message=on_message, on_close=on_close)

In [34]:
ws.run_forever()

False

In [ ]:
investment

[800.0,
 642.86,
 643.1742488380626,
 642.7069151601911,
 657.2239110392379,
 742.7793066640161,
 742.829799996891,
 742.9783497601941,
 743.0012621211536,
 742.9757837306947,
 742.9651537010534,
 757.1710542895512,
 771.2886130424192,
 785.6163837246652,
 914.1786062519457,
 728.3722301283051,
 728.3475679946731,
 728.4504238772545,
 728.5132505593004,
 728.402178708161,
 728.4624289234235,
 728.4872505498678,
 728.5753405194565,
 742.8879229544754,
 942.8707924012144,
 971.3887872354434,
 971.3586640612543,
 999.0047359310317,
 956.4542629597523,
 956.6362199484295,
 956.8282332862465,
 942.4335080844282,
 913.880272281792,
 913.7659367838392,
 899.4254191497025,
 885.0689368069554,
 870.7496850330818,
 856.4013499048527,
 999.2360772144192,
 984.758279522012,
 999.1283569414942,
 941.8191180123792,
 970.7161788278622,
 970.6856042182118,
 970.6796417192118,
 956.3589347028628,
 942.1131351829491,
 928.0738584035436,
 913.6181149100073,
 899.2094886626576,
 713.448484194335,
 713.480

In [ ]:
portfolio

0.0

In [ ]:
money_end

1004.3182235147877

In [ ]:
# Return and Comparison with Benchmark
beg = closes[0]
end = closes[-1]

btc_return = (end/beg-1)
btc_return*100

0.35039603281619147

In [ ]:
bot_return = (money_end/amount - 1)
bot_return*100

0.43182235147878334

In [ ]:
alpha = bot_return - btc_return
alpha*100

0.08142631866259187

In [ ]:
btc_risk = np.std(np.log(np.array(closes[1:]) / np.array(closes[:-1])))
btc_risk

0.0014330835166143855

In [ ]:
bot_risk = np.std(np.log(np.array(real_time_portfolio_value[1:]) / np.array(real_time_portfolio_value[:-1])))
bot_risk

0.0011890159285262599

In [ ]:
btc_sharpe_ratio = btc_return / btc_risk
round(btc_sharpe_ratio,3)

2.445

In [ ]:
bot_sharpe_ratio = bot_return_risk
round(bot_sharpe_ratio,3) / bot

3.632